![](https://cdn-images-1.medium.com/max/800/1*jcZLpgh3gppeFFgcpFSP0w.jpeg)

# End to End Pytorch DNN Walkthrough :

This notebook holds end to end Pytorch DNN walkthrough covering all the procedures which are needed to do for a competition.

# Content : 

## 1. Primary Visualization.
## 2. Understanding the solution.
## 3. Creating Dataset.
## 4. Creating Deep Neural Net.
## 5. Model Training.
## 6. Saving best Model.
## 7. Testing on testing data.
## 8. Creating submission.

# Importing supporting libraries : 

At first we need to import basic libraries that'll help us to find the visualize the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from glob import glob
import os
from PIL import Image

Now, we have to load the metadata and see whether that can be used for attributes or not.

In [ ]:
train_metadata = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_metadata.head()

In [ ]:
test_metadata = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_metadata.head()

In [ ]:
def show_images(folder_name, num_images = 9, shape = (3, 3)):
  row = shape[0]
  col = shape[1]
  assert num_images == row*col,"Total image number is not matching with the size..."
  fig, ax = plt.subplots(row, col, figsize = (20, 6))
  plt.suptitle(f"Images : {folder_name.split('/')[-2]}")
  for index in range(num_images):
    plt.subplot(row, col, index + 1)
    img = load_image(glob(f"{folder_name}/*jpg")[index])
    plt.imshow(img)
  plt.show()
def load_image(source):
  img = cv2.imread(source)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

show_images('../input/petfinder-pawpularity-score/train/')
show_images('../input/petfinder-pawpularity-score/test/', 4, (2, 2) )

In [ ]:
train_features = train_metadata.iloc[:,1:-1]
train_features.head()

In [ ]:
Y = train_metadata['Pawpularity']

Now checking through the **Pawpularity score**  if the data holds any outliers or not.

In [ ]:
sns.boxplot(Y)

In [ ]:
train_features.describe()

In [ ]:
train_features.shape

In [ ]:
for column in train_features.columns:
  print(f"{column} : \n{train_features[column].value_counts()}")

After performing basic EDA on the data now it is time to create the Pytorch dataset that'll generate the dataloader to make batches of data while training.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision.transforms import transforms
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import math
import time
from sklearn.metrics import r2_score
from termcolor import cprint
import warnings
warnings.filterwarnings('ignore')

As in some cases the images are horizontally different we are using the **horizontal flip augmentation** so that the data can be more general.

In [ ]:
class PawDataset(Dataset):
  def __init__(self, data_source, metadata, H = 128, W = 128, test_data = False):
    super(PawDataset, self).__init__()
    self.data_source = data_source
    self.metadata = metadata
    self.H = H
    self.W = W
    self.test_data = test_data
    self.augment = self.transform()

  def transform(self):
    augmentation = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
        ]
    )
    return augmentation

  def __len__(self):
    return len(self.metadata)

  def __getitem__(self, index):
    # image_link
    source = self.metadata['Id'][index]
    source = os.path.join(f"{self.data_source}{source}.jpg")
    # loading the image and tranforming it into a torh tensor
    image = self.load_image(source)
    # loading metadata
    metadata = self.metadata.iloc[index, 1:13].astype('float32').to_numpy().reshape(1,-1)
    if self.test_data == False:
        # target output
        image = self.augment(image)
        image = transforms.ToTensor()(image)
        target = self.metadata['Pawpularity'][index] / 100.0
        return (image, metadata, target)
    else:
        image = transforms.ToTensor()(image)
        return (image, metadata)
  def load_image(self, source):
    img = cv2.imread(source)
    img = cv2.resize(img, (self.H, self.W))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    return img

Batch size is taken 64 , so that it can be more generalized as well as more specific towards the training data.

In [ ]:
BATCH_SIZE = 64

In [ ]:
Train_ds = PawDataset('../input/petfinder-pawpularity-score/train/', train_metadata)
Train_dl = DataLoader(Train_ds, batch_size = BATCH_SIZE, shuffle = True)
train_size = int(0.8 * Train_ds.__len__())
val_size = Train_ds.__len__() - train_size
train_ds , val_ds = torch.utils.data.random_split(Train_ds, [train_size, val_size ])
train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True)
val_dl = DataLoader(val_ds, batch_size = BATCH_SIZE, shuffle = True)
test_ds = PawDataset('../input/petfinder-pawpularity-score/test/', test_metadata, test_data = True)
test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE, shuffle = False)
for patch, metadata, target in train_dl:
  print(patch.shape, metadata.shape, target.shape)
  break
for patch, metadata, target in val_dl:
  print(patch.shape, metadata.shape, target.shape)
  break
for patch, target in test_dl:
  print(patch.shape, target.shape)
  break

Now , we can see that the dataloaders are created and it is actually creating a batch of data.

Now, it's time to generate the Model class. This is the general purpose DNN generation process, and followed by every single researcher.

You can find the model graph [here](https://github.com/sagnik1511/Deep-Learning-Competitions/blob/main/Kaggle/Petfinder%20Pawpularity/assets/petfinder_model_graph.png).

In [ ]:
class cnn(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size = 3, stride = 1, padding = 0):
    super(cnn, self).__init__()
    self.cnn = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.ReLU(),
        nn.BatchNorm2d(out_channels, momentum = 0.95)
    )
  def forward(self, x):
    return self.cnn(x)

class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.fet_ext = nn.Sequential(
        cnn(3,4),
        cnn(4, 64),   # 128 -> 126
        nn.MaxPool2d(2), # 126 -> 63
        cnn(64, 32),   # 63 -> 61
        nn.MaxPool2d(2),  # 61 -> 30
        cnn(32, 16),  # 30 -> 28
        nn.MaxPool2d(2),
        nn.Dropout(0.15),
        nn.Flatten(), # 14*14*16
    )
    self.fc1 = nn.Sequential(
        nn.Linear(12, 64),
    )
    self.fc2 = nn.Sequential(
        nn.Linear(14*14*16, 1024),
        nn.Linear(1024, 64),
    )
    self.fc3 = nn.Sequential(
        nn.Linear(128, 1),
    )

  def forward(self, im_patch, mt_patch):
      mt_patch = mt_patch.squeeze(dim = 2)
      mt_patch = mt_patch.squeeze(dim = 1)
      cnn_op = self.fet_ext(im_patch)
      fc1_op = self.fc1(mt_patch)
      fc2_op = self.fc2(cnn_op)
      linear_op = torch.cat([fc1_op, fc2_op], axis = 1)
      output = self.fc3(linear_op)
      return output

As the training process can be slow using cpu , we are going to process the training inside the **GPU** itself.

In [ ]:
model = Network()
model = model.cuda()
model

In [ ]:
for name, param in model.named_parameters():
  print(f"{name} | {param.shape} | {param.dtype}")

Taking basic hyerparameters (This has been taken after long nmber of experiments).

In [ ]:
EPOCHS = 10
criterion = nn.MSELoss()
optm = optim.Adam(model.parameters(), lr = 1e-4)

Now, it's the most important process, we have to train the model, so that it can be well fitted as well as more generalized but overfitted.

In [ ]:
train_step_loss, val_step_loss = [], []
train_loss, val_loss = [], []
val_best_loss = np.inf
for epoch in range(EPOCHS):
  start_time = time.time()
  print(f"Epoch {epoch + 1} : ")
  epoch_loss = 0.0
  model.train()
  for index, (patch, metadata, target) in enumerate(train_dl):
    optm.zero_grad()
    patch = patch.float().cuda()
    metadata  = metadata.float().cuda()
    target = target.float().cuda()
    op = model(patch, metadata)
    loss = criterion(op, target)
    epoch_loss += loss.item() * patch.shape[0]
    train_step_loss.append(loss.item())
    if index % 10 == 9:
      print(f"step {index + 1} Loss: {'%.4f'%(loss.item())}")
    loss.backward()
    optm.step()
  epoch_loss /= train_size
  print(f"training data --> loss : {'%.4f'%(epoch_loss)}")
  train_loss.append(epoch_loss)
  model.eval()
  val_ep_loss = 0.0
  with torch.no_grad():
      for index, (patch, metadata, target) in enumerate(val_dl):
          patch = patch.float().cuda()
          metadata  = metadata.float().cuda()
          target = target.float().cuda()
          op = model(patch, metadata)
          loss = criterion(op, target)
          val_step_loss.append(loss.item())
          val_ep_loss += loss.item() * patch.shape[0]
  val_ep_loss /= val_size
  print(f"validation data --> loss : {'%.4f'%(val_ep_loss)}")
  val_loss.append(val_ep_loss)
  if val_ep_loss < val_best_loss :
    val_best_loss = val_ep_loss
    cprint("Success...Model Updated...", 'green')
    torch.save(model, 'best_model.pth')
  else:
    cprint("Failed... Model haven't uploaded...", 'red')
  elapsed_time = time.time() - start_time
  print(f"Elapsed time : {'%.2f'%(elapsed_time)} seconds...\n")
cprint("Training completed...", 'blue')

As the last saved model can have too much varinace with the validation data, we are going to use the best saved model so far.

In [ ]:
best_model = torch.load('./best_model.pth')

In [ ]:
best_model = best_model.cuda()

Now, the model is testing on the output data and one can see the data is getting out is in form of tensors, so we have to process that to fit inside the dataframe.

In [ ]:
with torch.no_grad():
    for index, (patch, metadata) in enumerate(test_dl):
        patch = patch.float().cuda()
        metadata  = metadata.float().cuda()
        op = best_model(patch, metadata)

Now, the final dataframe has been prepared, and ready to be submitted.

In [ ]:
sub_df = pd.DataFrame({'Id': test_metadata.Id, 'Pawpularity': op.squeeze(dim = 1).cpu().detach().numpy()})
sub_df

In [ ]:
sub_df.to_csv('submission.csv', index = False)